# <center> <img src="../../labs/img/ITESOLogo.png" alt="ITESO" width="480" height="130"> </center>
# <center> **Departamento de Electrónica, Sistemas e Informática** </center>
---
## <center> **Procesamiento de Datos Masivos** </center>
---
### <center> **Primavera 2025** </center>
---
### <center> **K means model** </center>

---
**Profesor**: Dr. Pablo Camarillo Ramirez

**Integrantes**:
- Lorena Ruelas Gaytán
- Yael Alejandro Rodríguez Barreto
- Ximena Isaac Horta
- Alberto Renteria Camacho

In [10]:
import findspark
findspark.init()

In [11]:
SPARK_ID = "9aa900e9e1bd"

#### Spark Conexion

In [12]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("MLSpark-K-means") \
    .master(f"spark://{SPARK_ID}:7077") \
    .config("spark.ui.port","4040") \
    .getOrCreate()
sc = spark.sparkContext
spark.conf.set("spark.sql.shuffle.partitions", "5")

#### Extraction of data

In [13]:
data_path = "/home/jovyan/notebooks/data/parquet/"

tweets_df = spark.read \
    .parquet(data_path)

print("Rows:", tweets_df.count(), "Cols:", len(tweets_df.columns))
tweets_df.show(10)
tweets_df.printSchema()

Rows: 3020 Cols: 11


+--------------------+-------+--------------------+--------------------+--------------------+--------------------+-------------+--------------+-----------+-----------+-----+
|            tweet_id|user_id|           timestamp|                text|            hashtags|            mentions|retweet_count|favorite_count|reply_count|quote_count|views|
+--------------------+-------+--------------------+--------------------+--------------------+--------------------+-------------+--------------+-----------+-----------+-----+
|e9ff298e-0593-403...|      3|2024-09-18T06:19:...|Id nesciunt elige...|["quod","voluptat...|        ["zhidalgo"]|          246|           892|         23|         17| 9767|
|bd456ef7-2eed-425...|      3|2024-09-20T12:59:...|Praesentium sint ...|                  []|  ["monteroarcelia"]|          246|           610|          4|         17|16917|
|2e9e17f3-1f8b-496...|      3|2025-01-30T01:07:...|Voluptates incidu...|      ["neque","in"]|["barriosgeorgina...|          412|  

### Assemble the features into a single vector column

In [14]:
from pyspark.ml.feature import VectorAssembler

numeric_cols = [
    "retweet_count", "favorite_count", "reply_count", "quote_count", "views"
]

assembler = VectorAssembler(inputCols=numeric_cols, outputCol="features")
assembled_df = assembler.transform(tweets_df)

---
# KMeans at the same time

#### Initialize KMeans

In [ ]:
from pyspark.ml.clustering import KMeans

k_values = [2, 5, 10, 15, 20]
kmeans = [KMeans().setK(k).setSeed(19) for k in k_values]

#### Training

In [ ]:
models = [k.fit(assembled_df) for k in kmeans]

#### Save models

In [ ]:
path_models = "/home/jovyan/notebooks/data/models/kmeans/"
#path_models = "/home/jovyan/notebooks/final_project/equipo/models/kmeans/"

for m, k in zip(models, k_values):
    m.save(f"{path_models}modelk{k}")

---
# See results of models

#### Read models

In [15]:
from pyspark.ml.clustering import KMeansModel

path_models = "/home/jovyan/notebooks/data/models/kmeans/"
#path_models = "/home/jovyan/notebooks/final_project/equipo/models/kmeans/"

models = []
k_values = [2, 5, 10, 15, 20]
for i in k_values:
    m = KMeansModel.load(f"{path_models}modelk{i}")
    models.append(m)
    print("Model", i, "ready")


Model 2 ready
Model 5 ready


Model 10 ready
Model 15 ready
Model 20 ready


#### Predictions

In [34]:
predictions = [model.transform(assembled_df) for model in models]
predictions[0].show()

+--------------------+-------+--------------------+--------------------+--------------------+--------------------+-------------+--------------+-----------+-----------+-----+--------------------+----------+
|            tweet_id|user_id|           timestamp|                text|            hashtags|            mentions|retweet_count|favorite_count|reply_count|quote_count|views|            features|prediction|
+--------------------+-------+--------------------+--------------------+--------------------+--------------------+-------------+--------------+-----------+-----------+-----+--------------------+----------+
|e9ff298e-0593-403...|      3|2024-09-18T06:19:...|Id nesciunt elige...|["quod","voluptat...|        ["zhidalgo"]|          246|           892|         23|         17| 9767|[246.0,892.0,23.0...|         1|
|bd456ef7-2eed-425...|      3|2024-09-20T12:59:...|Praesentium sint ...|                  []|  ["monteroarcelia"]|          246|           610|          4|         17|16917|[24

#### Save as CSV

In [35]:
from pyspark.sql.functions import col, split, regexp_replace

path_models = "/home/jovyan/notebooks/data/csvs/"
#path_models = "/home/jovyan/notebooks/final_project/equipo/models/kmeans/"

k_values = [2, 5, 10, 15, 20]
for i, p in zip(k_values, predictions):

    p = p.withColumn("features", col("features").cast("string"))
    p = p.withColumn("features", regexp_replace("features", r'^\[|\]$', ''))
    p = p.withColumn("features", split(col("features"), ","))

    num_features = len(p.select("features").first()[0])

    for j in range(num_features):
        p = p.withColumn(f"feature_{j}", col("features")[j].cast("double"))

    df = p.drop("features")
    df.coalesce(1).write.option("header", "true").csv(f"{path_models}csvwithk{i}")

ERROR:root:KeyboardInterrupt while sending command.                 (0 + 1) / 1]
Traceback (most recent call last):
  File "/opt/conda/spark-3.5.4-bin-hadoop3-scala2.13/python/lib/py4j-0.10.9.7-src.zip/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/conda/spark-3.5.4-bin-hadoop3-scala2.13/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
                          ^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/conda/lib/python3.11/socket.py", line 706, in readinto
    return self._sock.recv_into(b)
           ^^^^^^^^^^^^^^^^^^^^^^^
KeyboardInterrupt


KeyboardInterrupt: 

#### Evaluate model

In [ ]:
from pyspark.ml.evaluation import ClusteringEvaluator

for i in range(len(k_values)):
    print(f"\n K values = {k_values[i]}")
    evaluator = ClusteringEvaluator()
    silhouette = evaluator.evaluate(predictions[i])
    print(f"Silhouette score: {silhouette}")

In [ ]:
for i in range(len(k_values)):
    print(f"\n K values = {k_values[i]}")
    print("Cluster Centers: ")
    for center in models[i].clusterCenters():
        print(center)

In [ ]:
sc.stop()